# Census tract to Chicago community area aggregation by median income in the last 12 months

<b>Methodology:</b> [Linear interopolation](https://www.esri.com/arcgis-blog/products/arcgis-living-atlas/decision-support/medians) with binned medians.

In [50]:
# configs
import pandas as pd
import numpy as np

Downloaded 2023 5-year ACS data for B19001, Household income in the past 12 months binned, by census tract for Cook County

In [2]:
# load census data
acs = pd.read_csv('../census-data/ACSDT5Y2023.B19001_2025-07-11T175529/ACSDT5Y2023.B19001-Data.csv', skiprows=[1],
                  header=0,
                  usecols=['GEO_ID',
                           'B19001_001E', # total
                           'B19001_002E', # less than $10,000
                           'B19001_003E', # $10,000-14,999
                           'B19001_004E', # $15,000-19,999
                           'B19001_005E', # $20,000-24,999
                           'B19001_006E', # $25,000-29,999
                           'B19001_007E', # $30,000-34,999
                           'B19001_008E', # $35,000-39,999
                           'B19001_009E', # $40,000-44,999
                           'B19001_010E', # $45,000-49,999
                           'B19001_011E', # $50,000-59,999
                           'B19001_012E', # $60,000-74,999
                           'B19001_013E', # $75,000-99,999
                           'B19001_014E', # $100,000-124,999
                           'B19001_015E', # $125,000-149,999
                           'B19001_016E', # $150,000-199,999
                           'B19001_017E']) # $200,000 or more

In [4]:
acs.head(2)

,GEO_ID,B19001_001E,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E
0,1400000US17031010100,2190,125,109,41,148,118,191,51,45,96,102,299,418,82,118,124,123
1,1400000US17031010201,3038,223,61,153,0,92,172,355,243,233,49,192,576,220,201,161,107


In [5]:
# rename cols
acs.columns = ['geoid',
               'total',
               'less than $10,000',
               '$10,000-14,999',
               '$15,000-19,999',
               '$20,000-24,999',
               '$25,000-29,999',
               '$30,000-34,999',
               '$35,000-39,999',
               '$40,000-44,999',
               '$45,000-49,999',
               '$50,000-59,999',
               '$60,000-74,999',
               '$75,000-99,999',
               '$100,000-124,999',
               '$125,000-149,999',
               '$150,000-199,999',
               '$200,000 or more']

In [6]:
# clean id to match crosswalk tract geoid
acs['geoid'] = acs['geoid'].astype(str)
acs['geoid_clean'] = acs['geoid'].str.replace('1400000US', '')
acs['geoid_clean'] = acs['geoid_clean'].astype(int)

In [17]:
# create percent cols for each income bin
bin_columns = [
    'less than $10,000',
    '$10,000-14,999',
    '$15,000-19,999',
    '$20,000-24,999',
    '$25,000-29,999',
    '$30,000-34,999',
    '$35,000-39,999',
    '$40,000-44,999',
    '$45,000-49,999',
    '$50,000-59,999',
    '$60,000-74,999',
    '$75,000-99,999',
    '$100,000-124,999',
    '$125,000-149,999',
    '$150,000-199,999',
    '$200,000 or more'
]

# Create percent columns
for col in bin_columns:
    pct_col = f"pct {col}"
    acs[pct_col] = acs[col] / acs['total']

In [18]:
acs

,geoid,total,"less than $10,000","$10,000-14,999","$15,000-19,999","$20,000-24,999","$25,000-29,999","$30,000-34,999","$35,000-39,999","$40,000-44,999",...,"pct $35,000-39,999","pct $40,000-44,999","pct $45,000-49,999","pct $50,000-59,999","pct $60,000-74,999","pct $75,000-99,999","pct $100,000-124,999","pct $125,000-149,999","pct $150,000-199,999","pct $200,000 or more"
0,1400000US17031010100,2190,125,109,41,148,118,191,51,45,...,0.023288,0.020548,0.043836,0.046575,0.136530,0.190868,0.037443,0.053881,0.056621,0.056164
1,1400000US17031010201,3038,223,61,153,0,92,172,355,243,...,0.116853,0.079987,0.076695,0.016129,0.063199,0.189598,0.072416,0.066162,0.052995,0.035221
2,1400000US17031010202,1130,100,79,12,12,40,52,20,94,...,0.017699,0.083186,0.091150,0.092035,0.031858,0.208850,0.087611,0.027434,0.058407,0.040708
3,1400000US17031010300,3185,44,231,98,259,225,45,69,100,...,0.021664,0.031397,0.025432,0.091994,0.132496,0.136892,0.075981,0.076923,0.052119,0.071900
4,1400000US17031010400,2058,141,131,298,81,59,41,56,33,...,0.027211,0.016035,0.130224,0.049077,0.087464,0.105928,0.048105,0.062682,0.041302,0.067055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327,1400000US17031844600,851,36,87,83,12,48,9,28,0,...,0.032902,0.000000,0.000000,0.199765,0.019976,0.334900,0.000000,0.005875,0.042303,0.041128
1328,1400000US17031844700,683,74,90,0,27,33,30,0,12,...,0.000000,0.017570,0.014641,0.039531,0.060029,0.103953,0.144949,0.177160,0.021962,0.048316
1329,1400000US17031980000,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1330,1400000US17031980100,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# load cmap crosswalk
crosswalk = pd.read_csv('../census-data/Crosswalk_TR_to_CCA_2020 (1).csv')
crosswalk.head(2)

,TRACT,GEOID,CCA,TR_POP_RAT,TR_HH_RAT,TR_HU_RAT
0,17031010100,1,Rogers Park,1.0,1.0,1.0
1,17031010201,1,Rogers Park,1.0,1.0,1.0


In [32]:
# left merge crosswalk with census data so duplicate keys have same census info
merged = pd.merge(crosswalk, acs, left_on='TRACT', right_on='geoid_clean', how='left', indicator=True)

In [33]:
# check
merged[merged['TRACT'] == 17031843900]

,TRACT,GEOID,CCA,TR_POP_RAT,TR_HH_RAT,TR_HU_RAT,geoid,total,"less than $10,000","$10,000-14,999",...,"pct $40,000-44,999","pct $45,000-49,999","pct $50,000-59,999","pct $60,000-74,999","pct $75,000-99,999","pct $100,000-124,999","pct $125,000-149,999","pct $150,000-199,999","pct $200,000 or more",_merge
799,17031843900,42,Woodlawn,0.385979,0.312902,0.314274,1400000US17031843900,2148.0,120.0,307.0,...,0.032123,0.059125,0.018622,0.133613,0.125233,0.07635,0.049348,0.074953,0.001862,both
800,17031843900,43,South Shore,0.614021,0.687098,0.685726,1400000US17031843900,2148.0,120.0,307.0,...,0.032123,0.059125,0.018622,0.133613,0.125233,0.07635,0.049348,0.074953,0.001862,both


In [34]:
# are there na tracts? 
merged[merged['total'].isna()]

,TRACT,GEOID,CCA,TR_POP_RAT,TR_HH_RAT,TR_HU_RAT,geoid,total,"less than $10,000","$10,000-14,999",...,"pct $40,000-44,999","pct $45,000-49,999","pct $50,000-59,999","pct $60,000-74,999","pct $75,000-99,999","pct $100,000-124,999","pct $125,000-149,999","pct $150,000-199,999","pct $200,000 or more",_merge
805,17043840000,76,O'Hare,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
806,17043840801,76,O'Hare,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [35]:
# remove na tracts
merged = merged.dropna(subset=['total'])

In [36]:
# multiply total tract population by population ratio
merged['total'] = merged['total'].astype(int)
merged['est_total'] = merged['total'] * merged['TR_POP_RAT']

Calculate the estimated number of households in each income bin by multiplying the share of the income bin by the estimated total population.

In [38]:
# multiply est total by income bin pct breakdowns
for col in bin_columns:
    est_col = f"est {col}"
    merged[est_col] = merged[f"pct {col}"] * merged['est_total']

In [53]:
merged.head(2)

,TRACT,GEOID,CCA,TR_POP_RAT,TR_HH_RAT,TR_HU_RAT,geoid,total,"less than $10,000","$10,000-14,999",...,"est $35,000-39,999","est $40,000-44,999","est $45,000-49,999","est $50,000-59,999","est $60,000-74,999","est $75,000-99,999","est $100,000-124,999","est $125,000-149,999","est $150,000-199,999","est $200,000 or more"
0,17031010100,1,Rogers Park,1.0,1.0,1.0,1400000US17031010100,2190,125.0,109.0,...,51.0,45.0,96.0,102.0,299.0,418.0,82.0,118.0,124.0,123.0
1,17031010201,1,Rogers Park,1.0,1.0,1.0,1400000US17031010201,3038,223.0,61.0,...,355.0,243.0,233.0,49.0,192.0,576.0,220.0,201.0,161.0,107.0


In [59]:
# reshape merged as a long df with income bins on vertical axis
bin_columns = [
    "est less than $10,000",
    "est $10,000-14,999",
    "est $15,000-19,999",
    "est $20,000-24,999",
    "est $25,000-29,999",
    "est $30,000-34,999",
    "est $35,000-39,999",
    "est $40,000-44,999",
    "est $45,000-49,999",
    "est $50,000-59,999",
    "est $60,000-74,999",
    "est $75,000-99,999",
    "est $100,000-124,999",
    "est $125,000-149,999",
    "est $150,000-199,999",
    "est $200,000 or more"
]

merged.melt()

# Melt the dataframe
df_long = pd.melt(
    merged,  # your full dataframe
    id_vars=['TRACT', 'GEOID', 'CCA', 'total'],  # columns to keep
    value_vars=bin_columns,                      # columns to unpivot
    var_name='income_bin',                       # name of the new column holding bin labels
    value_name='households'                      # name of the new column holding values
)

df_long

,TRACT,GEOID,CCA,total,income_bin,households
0,17031010100,1,Rogers Park,2190,"est less than $10,000",125.000000
1,17031010201,1,Rogers Park,3038,"est less than $10,000",223.000000
2,17031010202,1,Rogers Park,1130,"est less than $10,000",100.000000
3,17031010300,1,Rogers Park,3185,"est less than $10,000",44.000000
4,17031010400,1,Rogers Park,2058,"est less than $10,000",141.000000
...,...,...,...,...,...,...
12875,17031843900,43,South Shore,2148,"est $200,000 or more",2.456083
12876,17031844600,38,Grand Boulevard,851,"est $200,000 or more",35.000000
12877,17031844700,29,North Lawndale,683,"est $200,000 or more",33.000000
12878,17031980000,76,O'Hare,0,"est $200,000 or more",NaN


In [77]:
# group df_long by cca

cca_income_dist = (
    df_long
    .groupby(['CCA', 'income_bin'], as_index=False)['households']
    .sum()
    .sort_values(['CCA', 'income_bin'])  
)

cca_income_dist

,CCA,income_bin,households
0,Albany Park,"est $10,000-14,999",531.000000
1,Albany Park,"est $100,000-124,999",2138.000000
2,Albany Park,"est $125,000-149,999",1153.000000
3,Albany Park,"est $15,000-19,999",323.000000
4,Albany Park,"est $150,000-199,999",1766.000000
...,...,...,...
1227,Woodlawn,"est $45,000-49,999",264.019367
1228,Woodlawn,"est $50,000-59,999",387.439171
1229,Woodlawn,"est $60,000-74,999",870.776050
1230,Woodlawn,"est $75,000-99,999",1095.828423


In [81]:
income_bounds = {
    'est less than $10,000': (0, 9999),
    'est $10,000-14,999': (10000, 14999),
    'est $15,000-19,999': (15000, 19999),
    'est $20,000-24,999': (20000, 24999),
    'est $25,000-29,999': (25000, 29999),
    'est $30,000-34,999': (30000, 34999),
    'est $35,000-39,999': (35000, 39999),
    'est $40,000-44,999': (40000, 44999),
    'est $45,000-49,999': (45000, 49999),
    'est $50,000-59,999': (50000, 59999),
    'est $60,000-74,999': (60000, 74999),
    'est $75,000-99,999': (75000, 99999),
    'est $100,000-124,999': (100000, 124999),
    'est $125,000-149,999': (125000, 149999),
    'est $150,000-199,999': (150000, 199999),
    'est $200,000 or more': (200000, 1000000)
}

# Map to new columns
cca_income_dist['lower_bound'] = cca_income_dist['income_bin'].map(lambda x: income_bounds[x][0])
cca_income_dist['upper_bound'] = cca_income_dist['income_bin'].map(lambda x: income_bounds[x][1])

cca_income_dist.sort_values(by=['CCA', 'lower_bound'], inplace=True)

cca_income_dist

,CCA,income_bin,households,lower_bound,upper_bound
15,Albany Park,"est less than $10,000",502.000000,0,9999
0,Albany Park,"est $10,000-14,999",531.000000,10000,14999
3,Albany Park,"est $15,000-19,999",323.000000,15000,19999
5,Albany Park,"est $20,000-24,999",631.000000,20000,24999
7,Albany Park,"est $25,000-29,999",748.000000,25000,29999
...,...,...,...,...,...
1230,Woodlawn,"est $75,000-99,999",1095.828423,75000,99999
1217,Woodlawn,"est $100,000-124,999",726.300600,100000,124999
1218,Woodlawn,"est $125,000-149,999",315.913803,125000,149999
1220,Woodlawn,"est $150,000-199,999",493.142662,150000,199999


In [83]:
def estimate_median_income(group):
    group = group.sort_values('lower_bound').reset_index(drop=True)
    group['cumulative'] = group['households'].cumsum()
    total = group['households'].sum()
    median_threshold = total / 2

    # find the bin where the cumulative sum exceeds median threshold
    for i, row in group.iterrows():
        if row['cumulative'] >= median_threshold:
            if i == 0:
                prev_cum = 0
            else:
                prev_cum = group.loc[i - 1, 'cumulative']

            bin_start = row['lower_bound']
            bin_end = row['upper_bound']
            bin_width = bin_end - bin_start
            bin_count = row['households']
            cum_before_bin = prev_cum

            # interpolation
            if bin_count == 0 or bin_width == 0:
                return bin_start  # avoid division by zero

            median = bin_start + ((median_threshold - cum_before_bin) / bin_count) * bin_width
            return median

    return None  

               CCA  estimated_median_income
0      Albany Park             82249.354058
1   Archer Heights             67397.234091
2    Armour Square             43502.615213
3          Ashburn             80909.449476
4   Auburn Gresham             45489.412587
..             ...                      ...
72       West Lawn             73410.708103
73    West Pullman             50429.644531
74      West Ridge             71524.002706
75       West Town            134132.870313
76        Woodlawn             35619.408577

[77 rows x 2 columns]


/var/folders/4l/t591xjt528jfczvfrdl8nnz40000gr/T/ipykernel_42717/349100341.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  median_by_cca = cca_income_dist.groupby('CCA').apply(estimate_median_income).reset_index()


In [85]:
# apply to each CCA
median_by_cca = cca_income_dist.groupby('CCA').apply(estimate_median_income).reset_index()
median_by_cca.columns = ['CCA', 'estimated_median_income']

median_by_cca

/var/folders/4l/t591xjt528jfczvfrdl8nnz40000gr/T/ipykernel_42717/147805904.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  median_by_cca = cca_income_dist.groupby('CCA').apply(estimate_median_income).reset_index()


,CCA,estimated_median_income
0,Albany Park,82249.354058
1,Archer Heights,67397.234091
2,Armour Square,43502.615213
3,Ashburn,80909.449476
4,Auburn Gresham,45489.412587
...,...,...
72,West Lawn,73410.708103
73,West Pullman,50429.644531
74,West Ridge,71524.002706
75,West Town,134132.870313


In [87]:
# export to csv
median_by_cca.to_csv('../processed/chicago_income_agg_community.csv', index=False)